Implement the Kalman filter with parameters $\mu, P, A, C, Q, R$. 

\begin{eqnarray}
x_0 & \sim & {\mathcal N}(\mu, P) \\
x_{t} & \sim & {\mathcal N}(A x_{t-1}, Q) \\
y_{t} & \sim & {\mathcal N}(C x_{t}, R) 
\end{eqnarray}

Known Parameters:

$\mu$ is $N \times 1$ 

$P$ is $N \times N$ and diagonal, positive semidefinite

$A$ is $N \times N$

$C$ is $M \times N$

$Q$ is $N \times N$ and diagonal, positive semidefinite

$R$ is $M \times M$ and diagonal, positive semidefinite

The input to the algorithm are the parameters and a sequence of 
observations $y_t$ for $t=1 \dots T$

$y_t$ is $M \times 1$ 

The output of the algorithm is for $t=1\dots T$

* A sequence of state predictions with

-- Predicted state mean estimate: $m_{t|t-1}$

-- Covariance Matrix: $\Sigma_{t|t-1}$

* A sequence of state estimates with

-- Filtered state estimate: $m_{t|t}$

-- Covariances: $\Sigma_{t|t}$

-- A sequence of loglikelihoods $l_k$

## Kalman Filter

The Kalman filtering algorithm is as follows:

### Initialization
The algorithm is initialized by
\begin{eqnarray}
m_{1|0} & = & \mu \\
\Sigma_{1|0} & = & P \\
l_0 & = & 0
\end{eqnarray}


For $t = 1\dots T $

### Update 
\begin{eqnarray}
S_t & = & C \Sigma_{t|t-1} C^\top + R \\
G_t & = & \Sigma_{t|t-1} C^\top S_t^{-1} \\
e_t & = & y_t - C m_{t|t-1} \\ 
m_{t|t} & = & m_{t|t-1} + G_t e_t \\ 
\Sigma_{t|t} & = & \Sigma_{t|t-1} - G_t C \Sigma_{t|t-1} \\ 
l_t & = & l_{t-1} - \frac{1}{2}\log |2\pi S_t| - \frac{1}{2} e_t^\top S_t^{-1} e_t 
\end{eqnarray}

### Prediction

\begin{eqnarray}
m_{t+1|t} & = & A m_{t|t} \\
\Sigma_{t+1|t} & = & A \Sigma_{t|t} A^\top + Q 
\end{eqnarray}

End For

## Your Task:

The loglikelihood computation seems to require the inversion of a large matrix and the computation of its determinant. We can cirmumvent this by 
a simple trick that works when the covariance matrix R is diagonal: we update the state by processing each entry of the observation incrementally.

We replace the update step with simpler operations where only scalar operations are needed.

Consider the observation matrix

\begin{eqnarray}
C & = & \left( \begin{array}{c} c_1 \\ c_2 \\ \vdots \\ c_M \end{array} \right)
\end{eqnarray}
where $c$ are row vectors. For example, if 
$$
C  =  \left( \begin{array}{ccc} 1 & 2 & 4 \\ 3 & 0 & 3 \end{array} \right)
$$
$c_1 = (1,2,4) $ and $c_2 = (3,0,3)$.

Similarly, assume that $R$ is diagonal with
$$
R = \left( \begin{array}{cccc} 
r_1 &     &  &  \\  
    & r_2 &  & \\ 
    & & \ddots  & \\ 
    &  &  & r_M  
\end{array} \right)
$$

 For example, if 
$$
R  =  \left( \begin{array}{ccc} 1 & 0 \\ 0 & 4 \end{array} \right)
$$
$r_1 = 1 $ and $r_2 = 4$.

The new algorithm is as follows
## Kalman Filter with incremental updates

### Initialization
The algorithm is initialized by
$$
m_{1|0}  = \mu 
$$
$$
\Sigma_{1|0}  =  P $$
$$
l_0  =  0
$$


For $t = 1\dots T $ (OuterFor) 
### Update


Define the temporary covariance and mean
$$
W_0 = \Sigma_{t|t-1} 
$$
$$
v_0 = m_{t|t-1} 
$$
$$
\lambda_0 = l_{t-1}
$$

For $i=1 \dots M$ (InnerFor)
$$
S_{t,i}  =  c_i W_{i-1} c_i^\top + r_i \\
$$
$$
G_{t,i}  =  \frac{1}{S_{t,i}} W_{i-1} c_i^\top  \\
$$
$$
e_{t,i}  =  y_t[m] - c_i v_{i-1} \\ 
$$
$$
v_i  =  v_{i-1} + G_{t,i} e_{t,i} \\ 
$$
$$
W_i  =  W_{i-1} - G_{t,i} c_i W_{i-1} \\ 
$$
$$
\lambda_i =  \lambda_{i-1} - \frac{1}{2}\log (2\pi S_{t,i}) - \frac{e_{t,i}^2}{2 S_{t,i}}  
$$
EndInnerfor 

\begin{eqnarray}
\Sigma_{t|t} & = & W_M \\
m_{t|t} & = & v_M \\
l_t & = & \lambda_M
\end{eqnarray}

### Prediction

\begin{eqnarray}
m_{t+1|t} & = & A m_{t|t} \\
\Sigma_{t+1|t} & = & A \Sigma_{t|t} A^\top + Q 
\end{eqnarray}

EndOuterFor

Your task is implementing the Kalman filter with incremental updates as an object in C++.

All the parameters will be specified when constructing the object and observations will be provided to the filter as a $M \times T$ matrix.

In [3]:
import numpy as np

N = 2
M = 3

A = np.matrix(np.ceil(5*np.random.randn(N,N)))
C = np.matrix(np.ceil(5*np.random.randn(M,N)))
R = np.matrix(2*np.eye(M))
Q = np.matrix(0.1*np.eye(N))

mu = np.matrix(10*np.ones((N,1)))
P = np.matrix(100*np.eye(N))

print('mu=\n',mu)
print('P=\n',P)

print('A=\n',A)
print('C=\n',C)
print('Q=\n',Q)
print('R=\n',R)

T = 10;

y = np.matrix(np.ceil(5*np.random.randn(M,T)))

print('observations=\n',y)

mu=
 [[ 10.]
 [ 10.]]
P=
 [[ 100.    0.]
 [   0.  100.]]
A=
 [[ 14.   6.]
 [  5.  11.]]
C=
 [[  4.   6.]
 [  3.   8.]
 [ 11. -12.]]
Q=
 [[ 0.1  0. ]
 [ 0.   0.1]]
R=
 [[ 2.  0.  0.]
 [ 0.  2.  0.]
 [ 0.  0.  2.]]
observations=
 [[  2.   5.  -2.  10.   2.   6.  -0.  -2.  -2.  -1.]
 [  7.  10.  -3.  -2.  -1.   2.   5.   3.  -8.   5.]
 [  5.  -0.   2.   3.   4.  -1.  -0.  -0.   2.   5.]]


Below is an (very bad numerically unstable) implementation of the Kalman filter for your reference. Nevertheless, for small $M, N, T$ the results would give an indication. Cross check with your implementation that your numerical results are about the same.



In [4]:

m_pred = mu
Sig_pred = P
l = 0

for t in range(T):
    S = C*Sig_pred*C.T + R 
    G = Sig_pred*C.T*(S.I)
    e = y[:,t] - C*m_pred
    m = m_pred + G*e
    Sig = Sig_pred - G*C*Sig_pred
    l = l - 0.5*np.log(np.linalg.det(2*np.pi*S)) - 0.5*e.T*S.I*e
    
    print('m_{}'.format(t+1), '=\n', m)
    print('Sig_{}'.format(t+1), '=\n',Sig)
    print('l_{}'.format(t+1), '=\n',l)
    print('\n')
    
    Sig_pred = A*Sig*A.T + Q
    mu_pred = A*mu


m_1 =
 [[ 0.74312123]
 [ 0.28941146]]
Sig_1 =
 [[ 0.01707879  0.0058791 ]
 [ 0.0058791   0.01021984]]
l_1 =
 [[-17.10272171]]


m_2 =
 [[ 0.75580554]
 [ 0.75994272]]
Sig_2 =
 [[ 0.01698656  0.0059101 ]
 [ 0.0059101   0.01014185]]
l_2 =
 [[-47.90276285]]


m_3 =
 [[-0.12902469]
 [-0.23464916]]
Sig_3 =
 [[ 0.01698575  0.00591073]
 [ 0.00591073  0.01014108]]
l_3 =
 [[-81.19746232]]


m_4 =
 [[ 0.59849414]
 [ 0.29338502]]
Sig_4 =
 [[ 0.01698574  0.00591074]
 [ 0.00591074  0.01014107]]
l_4 =
 [[-130.21808379]]


m_5 =
 [[ 0.2919432 ]
 [-0.02347952]]
Sig_5 =
 [[ 0.01698574  0.00591074]
 [ 0.00591074  0.01014107]]
l_5 =
 [[-163.7544587]]


m_6 =
 [[ 0.35633697]
 [ 0.43547743]]
Sig_6 =
 [[ 0.01698574  0.00591074]
 [ 0.00591074  0.01014107]]
l_6 =
 [[-196.02899145]]


m_7 =
 [[ 0.25143662]
 [ 0.30196306]]
Sig_7 =
 [[ 0.01698574  0.00591074]
 [ 0.00591074  0.01014107]]
l_7 =
 [[-229.33594073]]


m_8 =
 [[ 0.04978608]
 [ 0.11861284]]
Sig_8 =
 [[ 0.01698574  0.00591074]
 [ 0.00591074  0.01014107]]